In [34]:

import requests
import nltk
import re

from itertools import groupby

nltk.download('averaged_perceptron_tagger')
nltk.download('words')

# ---
def get_tag_that_is_chapter(tag): return tag.name == "h2" and tag.contents[0].name == "a"

def tokenize_text(book_text):
    TOKEN_PATTERN = r'\s+'
    regex_wt = nltk.RegexpTokenizer(pattern=TOKEN_PATTERN, gaps=True)
    word_tokens = regex_wt.tokenize(book_text)
    return word_tokens
# ---

# RAW TEXT VERSION

book_num = 17866
book_link = f"https://www.gutenberg.org/ebooks/{book_num}.txt.utf-8"
book_request = requests.get(book_link)

assert book_request.status_code == 200, f"Request of the book failed. Status: {book_request.status_code}"


book_chapters = re.split(r'Chapter [0-9]+', book_request.text, flags=re.IGNORECASE)[1:]
book_chapters[-1] = book_chapters[-1].split("End of Project Gutenberg")[0]

# Warning : indexes start at 1 !
book_chapters_paragraphs = dict()
tree_by_chapter = dict()

for i, chapter in enumerate(book_chapters):
    i = i+1 # Align with chapter num
    chapter_paragraphs = list()
    paragraphs = chapter.split('\r\n\r\n')
    for paragraph in paragraphs:
        clean_paragraph = re.sub(r"[\"\?\!\-\.\,\;\:\(\)\s]+", ' ', paragraph)
        chapter_paragraphs.append(clean_paragraph)

    book_chapters_paragraphs[i] = chapter_paragraphs

    # Tag and get the proper nouns
    tokens = ''.join(chapter_paragraphs)
    tokens = tokenize_text(tokens)
    tagged_tokens = nltk.pos_tag(tokens)
    
    # Group them if they are binded. Like "Lionel Lamy"
    groups = groupby(tagged_tokens, key=lambda x: x[1])
    names = [[w for w,_ in words] for tag,words in groups if tag=="NNP"]
    names = [" ".join(name) for name in names if len(name)>=2] 
    
    tree_by_chapter[i] = set(names)

# ---
# Tous les paragraphes (liste) d'un chapitre son accessible via :
# book_chapters_paragraphs.get(X) où X est le numéro de chapitre
# --
# Obtenir les noms propres détectés par chapitre :
# tree_by_chapter.get(X) où X est le numéro de chapitre
# ---

print(tree_by_chapter.get(4))

# TODO
# Il faut maintenant faire une vérification manuelle des personnages.. (voir le mail du prof)
# Je vais regarder comment implémenter l'algorithme de Louvain. 


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lioche\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lioche\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
{'Sam Browne', 'Fred Hines', "'Davy Crockett'", 'Mexican Mauser', 'Maginot Line', 'Yes Gresham', 'Whitneyville Walker', "Haven's Colt", "Jeff It's", 'Rifle Association', 'Lane Fleming', 'Virginia Manufactory', 'Walker Colt', 'World War', 'Ordinarily Lane', 'Stephen Rand', 'Rappahannock Forge', 'Premix Company', 'Paterson Colts', 'Germany A', 'Pierre Jarrett', "Lane Fleming's", 'Karen Lawrence Pierre', 'March Yes', "Colonel Walker's Texas Rangers", 'Elisha Collier', 'Cabot Joyner', 'Stephen Gresham', 'Colt Navy Models', 'Walpole Galleries', 'Humphrey Goode', 'Edison Public Power', 'U S North', 'Philip Cabot', 'Scott County', '